https://adventofcode.com/2023/day/22

In [86]:
from collections import deque
import numpy as np
import networkx as nx

In [2]:
with open("data/22.txt") as fh:
    data = fh.read()

In [3]:
testdata = """\
1,0,1~1,2,1
0,0,2~2,0,2
0,2,3~2,2,3
0,0,4~0,2,4
2,0,5~2,2,5
0,1,6~2,1,6
1,1,8~1,1,9
"""

In [4]:
def parse_puzzle(puzzle):
    L = []
    for line in puzzle.splitlines():
        astr, bstr = line.split("~")
        L.append(
            (
                tuple(int(a) for a in astr.split(",")),
                tuple(int(b) for b in bstr.split(",")),
            )
        )
    return L

In [5]:
[a for (a, b) in parse_puzzle(data) if a[0] > b[0] or a[1] > b[1] or a[2] > b[2]]

[]

In [95]:
def load_array(bricks):
    xmax = ymax = zmax = 0
    for _, (x2, y2, z2) in bricks:
        xmax = max(xmax, x2)
        ymax = max(ymax, y2)
        zmax = max(zmax, z2)
    ar = np.zeros((xmax + 1, ymax + 1, zmax + 1), dtype=int)
    for n, ((x1, y1, z1), (x2, y2, z2)) in enumerate(bricks, 1):
        ar[x1 : x2 + 1, y1 : y2 + 1, z1 : z2 + 1] = n
    return ar


def get_indices_for_value(ar, n):
    return list(zip(*np.nonzero(ar == n)))


def set_indices_to_value(ar, indices, n):
    for i in indices:
        ar[i] = n


def find_footprint(ar, n):
    xs, ys, zs = np.nonzero(ar == n)
    zmin = min(zs)
    return zmin, set((x, y) for (x, y, z) in zip(xs, ys, zs) if z == zmin)


def find_headprint(ar, n):
    xs, ys, zs = np.nonzero(ar == n)
    zmax = max(zs)
    return zmax, set((x, y) for (x, y, z) in zip(xs, ys, zs) if z == zmax)


def xys_for_z(ar, z):
    xs, ys = np.nonzero(ar[:, :, z])
    return set(zip(xs, ys))


def settle_block(ar, n):
    moves = 0
    indices = get_indices_for_value(ar, n)
    zmin = min(z for (x, y, z) in indices)
    footprint = set((x, y) for (x, y, z) in indices if z == zmin)
    while True:
        if zmin == 1:
            break
        headprint = xys_for_z(ar, zmin - 1)
        if footprint.intersection(headprint):
            break
        new_indices = [(x, y, z - 1) for (x, y, z) in indices]
        set_indices_to_value(ar, indices, 0)
        set_indices_to_value(ar, new_indices, n)
        indices = new_indices
        zmin -= 1
        moves += 1
    return moves


def settle_all(ar):
    moves = 1
    nstop = ar.max() + 1
    while moves:
        moves = sum(settle_block(ar, n) for n in range(1, nstop))


def supporting_blocks(ar, n):
    z0, footprint = find_footprint(ar, n)
    if z0 == 1:
        return None
    level_below = ar[:, :, z0 - 1]
    blocks_below = set(level_below.ravel())
    blocks_below.discard(0)
    supports = []
    for bb in blocks_below:
        z1, headprint = find_headprint(ar, bb)
        if z1 == z0 - 1 and headprint.intersection(footprint):
            supports.append(bb)
    return supports


def dont_disintegrate(ar):
    dd = set()
    for n in range(1, ar.max() + 1):
        sb = supporting_blocks(ar, n)
        if sb is not None and len(sb) == 1:
            dd.add(sb[0])
    return dd


def count_safe_to_disintegrate(puzzle):
    bricks = parse_puzzle(puzzle)
    ar = load_array(bricks)
    settle_all(ar)
    blockset = set(range(1, ar.max() + 1))
    dd = dont_disintegrate(ar)
    return len(blockset.difference(dd))

In [96]:
count_safe_to_disintegrate(testdata)

5

In [97]:
%%time
count_safe_to_disintegrate(data)
## Slow! Most of the time is in settle_all.

CPU times: user 6.71 s, sys: 0 ns, total: 6.71 s
Wall time: 6.71 s


434

### Part 2

In [87]:
%%time
blocks = parse_puzzle(data)
ar = load_array(blocks)
settle_all(ar)

CPU times: user 6.04 s, sys: 0 ns, total: 6.04 s
Wall time: 6.04 s


In [88]:
%%time
G = nx.DiGraph()
for n in range(1, ar.max() + 1):
    sbs = supporting_blocks(ar, n)
    if sbs is None:
        continue
    for sb in sbs:
        G.add_edge(sb, n)

CPU times: user 751 ms, sys: 0 ns, total: 751 ms
Wall time: 750 ms


In [89]:
def chain_reaction(G, n):
    keystones = {n}
    q = deque([n])
    while q:
        node = q.popleft()
        if keystones.issuperset(G.predecessors(node)):
            keystones.add(node)
        for successor in G.successors(node):
            q.append(successor)
    return len(keystones) - 1

In [90]:
%%time
sum(chain_reaction(G, n) for n in G)

CPU times: user 750 ms, sys: 0 ns, total: 750 ms
Wall time: 749 ms


61209